In [1]:
using Base.Math
using DataFrames
using LinearAlgebra
using LinearAlgebra.BLAS
import CSV

include("../data-visualization/GeometryConstants.jl")

10

In [2]:
input_file_path = "../data/shot_data_v0.1.csv"
output_file_path = "../data/shot_data_aug_v0.1.csv"

"../data/shot_data_aug_v0.1.csv"

In [3]:
sdf = DataFrame(CSV.File(input_file_path))

Row,location_x,location_y,body_part,outcome,shot_type
,Float64,Float64,String15,String31,String15
1,111.5,52.9,Right Foot,Off T,Open Play
2,113.9,26.4,Left Foot,Off T,Open Play
3,93.7,34.7,Left Foot,Saved,Open Play
4,109.2,39.1,Head,Off T,Open Play
5,107.8,24.7,Right Foot,Off T,Open Play
6,108.6,27.8,Left Foot,Wayward,Open Play
7,112.5,41.7,Head,Off T,Open Play
8,96.5,52.6,Left Foot,Post,Free Kick
9,112.4,41.0,Head,Off T,Open Play


In [4]:
df = sdf[sdf.shot_type .== "Open Play", :]

Row,location_x,location_y,body_part,outcome,shot_type
,Float64,Float64,String15,String31,String15
1,111.5,52.9,Right Foot,Off T,Open Play
2,113.9,26.4,Left Foot,Off T,Open Play
3,93.7,34.7,Left Foot,Saved,Open Play
4,109.2,39.1,Head,Off T,Open Play
5,107.8,24.7,Right Foot,Off T,Open Play
6,108.6,27.8,Left Foot,Wayward,Open Play
7,112.5,41.7,Head,Off T,Open Play
8,112.4,41.0,Head,Off T,Open Play
9,102.5,30.1,Right Foot,Saved,Open Play


In [5]:
"Find the angle subtended by the goal at the given position."
function angle_subtended(p_vec :: Vector{Float64}, l_vec :: Vector{Float64}, r_vec :: Vector{Float64}) :: Float64
    # using a . b == ||a|| ||b|| cos(theta) <=> cos(theta) = (a . b) / (||a|| ||b||)
    return rad2deg(acos(dot(l_vec, r_vec) / (nrm2(l_vec) * nrm2(r_vec))))
end

"Find the closest distance to goal from each shot location"
function distance_to_goal(p_vec, l_vec, r_vec)
    if (MAX_Y - GOAL_SIZE) / 2 <= p_vec[2] <= (MAX_Y + GOAL_SIZE) / 2
        return MAX_X - p_vec[1]
    end
    return min((√dot(l_vec, l_vec)), (√dot(r_vec, r_vec)))
end

distance_to_goal

In [6]:
angle_subtended_vec = []
distance_to_goal_vec = []
goal_post_edges = [(MAX_X, (MAX_Y - GOAL_SIZE) / 2), (MAX_X, (MAX_Y + GOAL_SIZE) / 2)]
for (x, y) in zip(df.location_x, df.location_y)
    p_vec = [x, y]
    l_vec = goal_post_edges[1] .- p_vec
    r_vec = goal_post_edges[2] .- p_vec
    push!(angle_subtended_vec, angle_subtended(p_vec, l_vec, r_vec))
    push!(distance_to_goal_vec, distance_to_goal(p_vec, l_vec, r_vec))
end

In [7]:
insertcols!(df, :distance => distance_to_goal_vec, :degrees_subtended => angle_subtended_vec)

Row,location_x,location_y,body_part,outcome,shot_type,distance,degrees_subtended
,Float64,Float64,String15,String31,String15,Any,Any
1,111.5,52.9,Right Foot,Off T,Open Play,12.3069,16.9826
2,113.9,26.4,Left Foot,Off T,Open Play,11.3741,13.3167
3,93.7,34.7,Left Foot,Saved,Open Play,26.3321,16.6444
4,109.2,39.1,Head,Off T,Open Play,10.8,40.4194
5,107.8,24.7,Right Foot,Off T,Open Play,16.6292,14.8953
6,108.6,27.8,Left Foot,Wayward,Open Play,14.0428,19.1384
7,112.5,41.7,Head,Off T,Open Play,7.5,54.2839
8,112.4,41.0,Head,Off T,Open Play,7.6,54.8817
9,102.5,30.1,Right Foot,Saved,Open Play,18.4678,19.8284


In [8]:
CSV.write(output_file_path, df)

"../data/shot_data_aug_v0.1.csv"